In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import numpy as np

In [3]:

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mffwpIhBU5Nrj1KFmGuCXS3fGPPcIN47' -O data.csv
df = pd.read_csv('data.csv')
df

--2021-04-29 21:28:29--  https://docs.google.com/uc?export=download&id=1mffwpIhBU5Nrj1KFmGuCXS3fGPPcIN47
Resolving docs.google.com (docs.google.com)... 172.217.13.238, 2607:f8b0:4004:803::200e
Connecting to docs.google.com (docs.google.com)|172.217.13.238|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/103tl17sv2fdjrl30cl8olqggvkr0nsa/1619731650000/09424851040513535275/*/1mffwpIhBU5Nrj1KFmGuCXS3fGPPcIN47?e=download [following]
--2021-04-29 21:28:31--  https://doc-0o-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/103tl17sv2fdjrl30cl8olqggvkr0nsa/1619731650000/09424851040513535275/*/1mffwpIhBU5Nrj1KFmGuCXS3fGPPcIN47?e=download
Resolving doc-0o-3g-docs.googleusercontent.com (doc-0o-3g-docs.googleusercontent.com)... 172.217.15.65, 2607:f8b0:4004:800::2001
Connecting to doc-0o-3g-docs.googleusercontent.com (doc-0o-3g-docs.google

,Unnamed: 0,Text,Label
0,0,dont think im get babi white two white j nike...,normal
1,1,cannot continu call feminist right womxn arent...,normal
2,2,nawt yall nigger ignor,normal
3,3,bit confus coz chines ppl access twitter thn c...,hatespeech
4,4,bitch whataburg eat burger top bun hold palm h...,hatespeech
...,...,...,...
20143,20143,ur still twitter tell carlton said alcohol dru...,offensive
20144,20144,first got said hate trump consid troll peopl y...,offensive
20145,20145,macht der moslem wenn der zion gegen seinen pr...,normal
20146,20146,aw look world demograph asian fuck everywher a...,hatespeech


In [4]:
DropDataNumber = 16000

df.drop(df.tail(DropDataNumber).index,
        inplace = True)
df

,Unnamed: 0,Text,Label
0,0,dont think im get babi white two white j nike...,normal
1,1,cannot continu call feminist right womxn arent...,normal
2,2,nawt yall nigger ignor,normal
3,3,bit confus coz chines ppl access twitter thn c...,hatespeech
4,4,bitch whataburg eat burger top bun hold palm h...,hatespeech
...,...,...,...
4143,4143,rofl hawaii explod california fall pacif hilla...,hatespeech
4144,4144,morn women push call faggot push way continu w...,offensive
4145,4145,porch monkey,normal
4146,4146,one orang shirt syrian refuge,normal


In [5]:
#metrics=[tf.keras.metrics.AUC()]
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [6]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['Text']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(Text):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_Text = []
    for Text in Text:
        tokens = Text.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_Text.append(word2vec_embedding)
    return encoded_Text
        
def get_padded_encoded_Text(encoded_Text):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_Text_encoding = []
    for enc_review in encoded_Text:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_Text_encoding.append(enc_review)
    return padded_Text_encoding

def sentiment_encode(Label):
    """
    return one hot encoding for Y value
    """
    if Label == 'normal':
        return [1,0,0]
    elif Label == 'hatespeech':
        return [0,1,0]
    else:
        return [0,0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    Text = df['Text'].tolist()
    
    encoded_Text = get_word2vec_enc(Text)
    padded_encoded_Text = get_padded_encoded_Text(encoded_Text)
    # encoded sentiment
    sentiments = df['Label'].tolist()
    encoded_sentiment = [sentiment_encode(Label) for Label in sentiments]
    X = np.array(padded_encoded_Text)
    Y = np.array(encoded_sentiment)
    return X, Y 

In [7]:
df.astype
df.dtypes

df = df[df['Text'].notnull()]

# Preprocess (encode text to number)

In [8]:
# movie_reviews_train = [
#          {'review': 'this is the best movie', 'sentiment': 'positive'},
#          {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
#          {'review': 'it was waste of money and time', 'sentiment': 'negative'},
#          {'review': 'the worst movie ever', 'sentiment': 'negative'}
#     ]
# df = pd.DataFrame(movie_reviews_train)

# max_length is used for max sequence of input
max_length = get_max_length(df)

X, y= preprocess(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
X_train

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.04766034, -0.05748302,  0.03545095, ...,  0.00521918,
         -0.0722433 ,  0.01327429],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.08465096, -0.03669273,  0.0231047 , ..., -0.00519624,
         -0.05797724, -0.02840648]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.01862679, -0.0889974 , -0.06428857, ...,  

# Build Model

In [103]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
from tensorflow.keras import regularizers
model.add(Dense(3, activation='softmax'))

model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7febf6118fd0>>

In [116]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
             #,metrics=[tf.keras.metrics.AUC()]
             )

# Train

In [117]:
print('Train...')
model.fit(X_train, y_train,epochs=100,
          validation_data=(X_test,y_test),
          batch_size=32)

Train...
Epoch 1/100
87/87 [==============================] - 4s 19ms/step - loss: 0.6315 - accuracy: 0.7613 - auc_9: 0.8921 - recall_3: 0.7312 - precision_3: 0.7833 - val_loss: 1.3884 - val_accuracy: 0.6384 - val_auc_9: 0.7944 - val_recall_3: 0.6026 - val_precision_3: 0.6648
Epoch 2/100
87/87 [==============================] - 1s 11ms/step - loss: 0.6106 - accuracy: 0.7663 - auc_9: 0.8955 - recall_3: 0.7394 - precision_3: 0.7895 - val_loss: 1.3358 - val_accuracy: 0.6238 - val_auc_9: 0.7933 - val_recall_3: 0.5931 - val_precision_3: 0.6517
Epoch 3/100
87/87 [==============================] - 1s 11ms/step - loss: 0.6093 - accuracy: 0.7567 - auc_9: 0.8995 - recall_3: 0.7340 - precision_3: 0.7931 - val_loss: 1.5390 - val_accuracy: 0.6472 - val_auc_9: 0.7975 - val_recall_3: 0.6150 - val_precision_3: 0.6846
Epoch 4/100
87/87 [==============================] - 1s 11ms/step - loss: 0.5911 - accuracy: 0.7667 - auc_9: 0.9022 - recall_3: 0.7442 - precision_3: 0.7969 - val_loss: 1.8678 - val_accur

In [118]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 32)                36224     
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 264       
_________________________________________________________________
dropout_14 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 72        
_________________________________________________________________
dropout_15 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                

# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [119]:

test_df = df

test_X, test_Y = preprocess(test_df)
test_X

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

130/130 - 1s - loss: 1.8532 - accuracy: 0.7982 - auc_9: 0.9056 - recall_3: 0.7553 - precision_3: 0.8212


ValueError: ignored

In [ ]:
test_df

In [ ]:
df.dtypes


In [ ]:
def value_transform(x):
  if x<0.5:
    x=0
  else:
    x=1
  return x

In [ ]:
# df['Text'] = np.asarray(df['Text']).astype(np.int64)
preds = model.predict(test_X)

In [ ]:
preds = np.array([[value_transform(j) for j in preds[i]] for i in range(20148 - DropDataNumber)], dtype='int32')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
preds.ndim

In [ ]:
preds.shape

In [ ]:
test_Y.shape

In [ ]:
accuracy_score(test_Y.argmax(axis=1), preds.argmax(axis=1))

In [ ]:
confusion_matrix(test_Y.argmax(axis=1), preds.argmax(axis=1))